# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [3]:
100 * len(student_data.loc[(student_data.passed=="yes")])/float(len(student_data))


67.0886075949367

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = len(student_data)
n_features = len(student_data.columns)
n_passed = len(student_data.loc[(student_data.passed=="yes")])
n_failed = len(student_data.loc[(student_data.passed=="no")])
grad_rate = 100 * len(student_data.loc[(student_data.passed=="yes")])/float(len(student_data))
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 31
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [7]:
from sklearn.cross_validation import StratifiedShuffleSplit

# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

cv = StratifiedShuffleSplit(student_data['passed'], 1, test_size  = num_test, random_state = 42)


# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
for train_index, test_index in cv:
    X_train = X_all.iloc[train_index]
    y_train = y_all.iloc[train_index]
    X_test = X_all.iloc[test_index]
    y_test = y_all.iloc[test_index]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [16]:
from sklearn.metrics import f1_score
def train_predict(clf, X_train, y_train, X_test, y_test, s, debug = False):
    if debug: 
        print "Training {}...".format(clf.__class__.__name__)
   
    start = time.time()
    clf.fit(X_train[:s], y_train[:s])    
    train_time = time.time() - start
    if debug: 
        print "Done!\nTraining time (secs): {:.3f}".format(train_time)

    start = time.time()
    train_predictions = clf.predict(X_train[:s])    
    timeToPredOnTrainData = time.time() - start
    if debug:
        print "Done!\nPrediction time (secs): {:.3f}".format(timeToPredOnTrainData)
   
        
    start = time.time()
    #keeping test set constant
    test_predictions = clf.predict(X_test)
    timeToPredOnTestData = time.time() - start
    if debug:    
        print "Done!\nPrediction time (secs): {:.3f}".format(timeToPredOnTestData)        
    
    trainF1 =  f1_score(train_predictions, y_train[:s], pos_label='yes')
    testF1 =  f1_score(test_predictions, y_test, pos_label='yes')
    
    return train_time, timeToPredOnTrainData, timeToPredOnTestData, trainF1, testF1

In [91]:
import time

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

#Ensemble methods
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

listOfModels = []

#These are my 3 chosen models - create them first to be easy to filter data frame latter on
listOfModels.append(KNeighborsClassifier())
listOfModels.append(svm.SVC())
listOfModels.append(GaussianNB())

listOfModels.append(ExtraTreesClassifier())
listOfModels.append(RandomForestClassifier())
listOfModels.append(DecisionTreeClassifier())
listOfModels.append(AdaBoostClassifier())
listOfModels.append(GradientBoostingClassifier())



sizes = [100,200,300]

mycolumns = ['Model', 'TrainingSize', 'TrainTime', 'PredOnTrainTime', 'PredOnTestTime', 'TrainingF1Score', 'TestingF1Score']
resultdf = pd.DataFrame(columns=mycolumns)

for clf in listOfModels:
    for i, s in enumerate(sizes):    
        a,b,c,d,e = train_predict(clf, X_train, y_train, X_test, y_test,s)
        resultdf.loc[len(resultdf)] = clf.__class__.__name__, s, a,b,c,d,e

In [93]:
#resultdf
from IPython.display import display, HTML
for clf in listOfModels[:3]:
    display(resultdf.loc[resultdf.Model == clf.__class__.__name__].sort(['TestingF1Score'], ascending = [0]) )

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Model,TrainingSize,TrainTime,PredOnTrainTime,PredOnTestTime,TrainingF1Score,TestingF1Score
2,KNeighborsClassifier,300,0.002,0.008,0.002,0.853933,0.813793
1,KNeighborsClassifier,200,0.001,0.003,0.002,0.809689,0.785714
0,KNeighborsClassifier,100,0.001,0.001,0.002,0.825175,0.758621


,Model,TrainingSize,TrainTime,PredOnTrainTime,PredOnTestTime,TrainingF1Score,TestingF1Score
4,SVC,200,0.004,0.003,0.002,0.843137,0.810458
5,SVC,300,0.008,0.005,0.002,0.866379,0.805195
3,SVC,100,0.002,0.001,0.001,0.835443,0.802548


,Model,TrainingSize,TrainTime,PredOnTrainTime,PredOnTestTime,TrainingF1Score,TestingF1Score
8,GaussianNB,300,0.001,0.001,0.000,0.813397,0.776119
7,GaussianNB,200,0.002,0.000,0.001,0.805970,0.721805
6,GaussianNB,100,0.001,0.001,0.000,0.775194,0.645669


In [92]:
resultdf.sort(['TestingF1Score','TrainingF1Score'], ascending = [0,0])
#resultdf.to_csv('resultdf.csv',index=False) 

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,Model,TrainingSize,TrainTime,PredOnTrainTime,PredOnTestTime,TrainingF1Score,TestingF1Score
2,KNeighborsClassifier,300,0.002,0.008,0.002,0.853933,0.813793
4,SVC,200,0.004,0.003,0.002,0.843137,0.810458
5,SVC,300,0.008,0.005,0.002,0.866379,0.805195
3,SVC,100,0.002,0.001,0.001,0.835443,0.802548
1,KNeighborsClassifier,200,0.001,0.003,0.002,0.809689,0.785714
8,GaussianNB,300,0.001,0.001,0.000,0.813397,0.776119
11,ExtraTreesClassifier,300,0.026,0.002,0.001,1.000000,0.765957
14,RandomForestClassifier,300,0.024,0.002,0.001,0.987593,0.764706
0,KNeighborsClassifier,100,0.001,0.001,0.002,0.825175,0.758621
19,AdaBoostClassifier,200,0.107,0.007,0.007,0.890511,0.744526


In [ ]:
(the x axis was set on a logaritmic scale from 0.6 to 1.3 to make visualization easier)
(sorted by the higher test f1 score )

In [149]:
#sorted(eclf.get_params().keys())

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.grid_search import GridSearchCV
clf1 = KNeighborsClassifier(n_jobs = -1)
clf2 = svm.SVC(probability=True) #probability=True is necessary for the soft voting 
clf3 = GaussianNB()
clf4 = RandomForestClassifier()

eclf = VotingClassifier(estimators=
                        [
                            ('knn', clf1),
                            ('svm', clf2),
                            ('gnb', clf3),
                            ('rfc', clf4),
                        ], voting='soft')


params = {'knn__n_neighbors': [4,5,6],
          'knn__weights'   : ['uniform','distance'],
          'knn__algorithm' : ['ball_tree', 'kd_tree', 'kd_tree', 'brute'],
          'knn__leaf_size' : [10,20,30,35],
          'knn__metric'    : ['minkowski', 'manhattan', 'euclidean'],
         
          'svm__kernel'  : ['linear', 'rbf'],
          'svm__gamma': [1e-3, 1e-4],
          'svm__C': [0.01, 0.1,1, 10],
          
          'rfc__n_estimators' : [10,20,30],
          'rfc__max_features' : [5,7,10,15],
          'rfc__min_samples_split' : [2,4,6] 
         }
        
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_all, y_all)
print grid.best_estimator_

In [109]:
clf3

GaussianNB()

In [73]:
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = GaussianNB()
eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')

params = {'lr__C': [1.0, 100.0], 'rf__n_estimators': [10, 200],}

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5)
grid = grid.fit(X_all, y_all)

In [78]:
grid.best_estimator_


VotingClassifier(estimators=[('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform'))],
         voting='hard', weights=None)

In [47]:
clf1

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [48]:
clf2

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [49]:
clf3

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

In [ ]:
# TODO: Fine-tune your model and report the best F1 score